In [105]:
import pandas as pd
import numpy as np
from keras.datasets import mnist
from keras.models import load_model
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from datetime import datetime as dt

In [106]:
train = pd.read_csv("train.csv")
target = train['label']
train = train.drop("label", axis=1)
test = pd.read_csv("test.csv")

In [107]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [108]:
def reshape_28(array):
    array = array.reshape((len(array),28,28,1))
    return array

In [109]:
train = reshape_28(train.values)
X_train = reshape_28(X_train)
X_test = reshape_28(X_test)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
target = to_categorical(target)

In [110]:
test = reshape_28(test.values.astype("float32") / 255)

In [111]:
X = np.concatenate([X_train, X_test, train], axis=0)
X = X.astype("float32") / 255
y = np.concatenate([y_train, y_test, target], axis=0)

In [112]:
stop = EarlyStopping(monitor="val_loss", patience=2)

In [113]:
print(f"Model run at {dt.now()}.")

Model run at 2022-01-10 11:52:56.111539.


In [114]:
model = load_model("MNIST_11022.hdf5")
model.fit(X, y, epochs=6, verbose=2, batch_size=256, validation_split=0.2, callbacks=[stop])

Epoch 1/6
350/350 - 44s - loss: 0.0014 - accuracy: 0.9996 - val_loss: 0.0068 - val_accuracy: 0.9985
Epoch 2/6
350/350 - 43s - loss: 0.0049 - accuracy: 0.9986 - val_loss: 0.0031 - val_accuracy: 0.9990
Epoch 3/6
350/350 - 43s - loss: 0.0021 - accuracy: 0.9993 - val_loss: 9.8627e-04 - val_accuracy: 0.9996
Epoch 4/6
350/350 - 44s - loss: 0.0018 - accuracy: 0.9994 - val_loss: 0.0030 - val_accuracy: 0.9992
Epoch 5/6
350/350 - 45s - loss: 0.0028 - accuracy: 0.9993 - val_loss: 0.0030 - val_accuracy: 0.9991


In [123]:
predictions = pd.DataFrame(model.predict(test)).idxmax(axis=1)

In [125]:
submission = pd.DataFrame({"ImageId":np.arange(1,28001), "label":predictions})

In [126]:
submission.to_csv("MNIST_11022.csv", index=False)